#ETL

## Importing libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
! pip install fuzzywuzzy
! pip install pyarrow

ERROR: unknown command "instal" - maybe you meant "install"


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import pyarrow as pa
import pyarrow.parquet as pq
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

## Importing csv

In [ ]:
cities = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vQVzyKZ9KHcOfW0uZwqUszAdI-opC72lYEAtpJBZxwuWTuvc_h18SIaCAgwvxXfHt2ixlraZCcs0Lrl/pub?output=csv')

In [ ]:
sales = pd.read_csv("/content/drive/Shareddrives/Projeto II/CSV_raw/sales.csv")

<ipython-input-45-1883645e054f>:1: DtypeWarning: Columns (11,13) have mixed types. Specify dtype option on import or set low_memory=False.
  sales = pd.read_csv("/content/drive/Shareddrives/Projeto II/CSV_raw/sales.csv")


In [ ]:
product = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vTLVeXZeo0CrX_IoDUEgASMnqqfxSMLpt_tB_7dxOWRzXrYBJVKUEpA_lUmA2JrGw/pub?output=csv")

In [ ]:
fore_rev = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vQcchFjMPX1jweLzxW74vmshHbw0afm3VKEPffmsZCtHmduzWTcrO9T6HVLKJVXUw/pub?output=csv")

##Cities

In [ ]:
cities.head()
cities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63 entries, 0 to 62
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   store_id      63 non-null     object
 1   storetype_id  63 non-null     object
 2   store_size    63 non-null     int64 
 3   city_id_old   63 non-null     object
 4   country_id    63 non-null     object
 5   city_code     63 non-null     object
dtypes: int64(1), object(5)
memory usage: 3.1+ KB


### Replace encoding errors

In [ ]:
# Incorrectly encoded city codes
incorrect_city_names = cities.city_code

# List of correct city codes
correct_city_names = ['Istanbul', 'Antalya', 'Sanliurfa', 'Konya', 'Izmir',
                    'Samsun', 'Kahramanmaras', 'Van', 'Denizli', 'Erzurum',
                    'Adana', 'Gaziantep', 'Diyarbakir', 'Kayseri', 'Bursa',
                    'Mersin', 'Ankara', 'Adapazari', 'Eskisehir']

In [ ]:
# Function to find the best match for the incorrect code in the list of correct codes
def correct_city_name(incorrect_name, correct_names):
    best_match = process.extractOne(incorrect_name, correct_names, scorer=fuzz.ratio)
    return best_match[0] if best_match else incorrect_name


In [ ]:
# Replace incorrect city code with correct ones
corrected_city_names = [correct_city_name(name, correct_city_names) for name in incorrect_city_names]

# Print unique correct city code
set(corrected_city_names)

{'Adana',
 'Adapazari',
 'Ankara',
 'Antalya',
 'Bursa',
 'Denizli',
 'Diyarbakir',
 'Erzurum',
 'Eskisehir',
 'Gaziantep',
 'Istanbul',
 'Izmir',
 'Kahramanmaras',
 'Kayseri',
 'Konya',
 'Mersin',
 'Samsun',
 'Sanliurfa',
 'Van'}

### Create a df with the correct city_code

In [ ]:
cities_cleaned = cities.assign(city_code=corrected_city_names)

### Comparing original vs cleaned

In [ ]:
cities.head()

,store_id,storetype_id,store_size,city_id_old,country_id,city_code
0,S0036,ST04,21,C001,TURKEY,Denizli
1,S0005,ST04,19,C001,TURKEY,Denizli
2,S0104,ST04,47,C002,TURKEY,Ankara
3,S0068,ST03,14,C003,TURKEY,?zmir
4,S0086,ST03,12,C003,TURKEY,?zmir


In [ ]:
cities_cleaned.head()

,store_id,storetype_id,store_size,city_id_old,country_id,city_code
0,S0036,ST04,21,C001,TURKEY,Denizli
1,S0005,ST04,19,C001,TURKEY,Denizli
2,S0104,ST04,47,C002,TURKEY,Ankara
3,S0068,ST03,14,C003,TURKEY,Izmir
4,S0086,ST03,12,C003,TURKEY,Izmir


### Convert country_id to capitalize only the first letter

In [ ]:
cities_cleaned['country_id'] = cities_cleaned['country_id'].str.capitalize()

###Creating cities_cleaned.csv

In [ ]:
# save cities cleaned

# cities_cleaned.to_csv('cities_cleaned.csv', index=False)
# cities_cleaned.to_parquet('cities_cleaned.parquet', engine='pyarrow', index=False)


##Products

In [ ]:
product.head()

,product_id,product_length,product_depth,product_width,cluster_id,hierarchy1_id,hierarchy2_id,hierarchy3_id,hierarchy4_id,hierarchy5_id
0,P0000,5.0,20.0,12.0,NaN,H00,H0004,H000401,H00040105,H0004010534
1,P0001,13.5,22.0,20.0,cluster_5,H01,H0105,H010501,H01050100,H0105010006
2,P0002,22.0,40.0,22.0,cluster_0,H03,H0315,H031508,H03150800,H0315080028
3,P0004,2.0,13.0,4.0,cluster_3,H03,H0314,H031405,H03140500,H0314050003
4,P0005,16.0,30.0,16.0,cluster_9,H03,H0312,H031211,H03121109,H0312110917


In [ ]:
product.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 699 entries, 0 to 698
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   product_id      699 non-null    object 
 1   product_length  681 non-null    float64
 2   product_depth   683 non-null    float64
 3   product_width   683 non-null    float64
 4   cluster_id      649 non-null    object 
 5   hierarchy1_id   699 non-null    object 
 6   hierarchy2_id   699 non-null    object 
 7   hierarchy3_id   699 non-null    object 
 8   hierarchy4_id   699 non-null    object 
 9   hierarchy5_id   699 non-null    object 
dtypes: float64(3), object(7)
memory usage: 54.7+ KB


In [ ]:
# Change data types
product_cleaned = pd.DataFrame(product)
hierarchy_columns = ['hierarchy1_id','hierarchy2_id', 'hierarchy3_id', 'hierarchy4_id', 'hierarchy5_id']
product_cleaned[hierarchy_columns] = product[hierarchy_columns].astype('category')

product_cleaned.info()
product_cleaned.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 699 entries, 0 to 698
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   product_id      699 non-null    object  
 1   product_length  681 non-null    float64 
 2   product_depth   683 non-null    float64 
 3   product_width   683 non-null    float64 
 4   cluster_id      649 non-null    object  
 5   hierarchy1_id   699 non-null    category
 6   hierarchy2_id   699 non-null    category
 7   hierarchy3_id   699 non-null    category
 8   hierarchy4_id   699 non-null    category
 9   hierarchy5_id   699 non-null    category
dtypes: category(5), float64(3), object(2)
memory usage: 52.2+ KB


,product_id,product_length,product_depth,product_width,cluster_id,hierarchy1_id,hierarchy2_id,hierarchy3_id,hierarchy4_id,hierarchy5_id
0,P0000,5.0,20.0,12.0,NaN,H00,H0004,H000401,H00040105,H0004010534
1,P0001,13.5,22.0,20.0,cluster_5,H01,H0105,H010501,H01050100,H0105010006
2,P0002,22.0,40.0,22.0,cluster_0,H03,H0315,H031508,H03150800,H0315080028
3,P0004,2.0,13.0,4.0,cluster_3,H03,H0314,H031405,H03140500,H0314050003
4,P0005,16.0,30.0,16.0,cluster_9,H03,H0312,H031211,H03121109,H0312110917


In [ ]:
# save products cleaned

# product_cleaned.to_csv('product_cleaned.csv', index=False)
# product_cleaned.to_parquet('product_cleaned.parquet', engine='pyarrow', index=False)

#read parquet file
# product_cleaned = pd.read_parquet('/content/drive/Shareddrives/Projeto II/Created_Tables/product_cleaned.parquet')
# product_cleaned = product_cleaned.to_pandas()
# product_cleaned
# product_cleaned.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 699 entries, 0 to 698
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   product_id      699 non-null    object  
 1   product_length  681 non-null    float64 
 2   product_depth   683 non-null    float64 
 3   product_width   683 non-null    float64 
 4   cluster_id      649 non-null    object  
 5   hierarchy1_id   699 non-null    category
 6   hierarchy2_id   699 non-null    category
 7   hierarchy3_id   699 non-null    category
 8   hierarchy4_id   699 non-null    category
 9   hierarchy5_id   699 non-null    category
dtypes: category(5), float64(3), object(2)
memory usage: 52.2+ KB


##Sales

In [ ]:
sales.head()

,Unnamed: 0,store_id,product_id,date,sales,revenue,stock,price,promo_type_1,promo_bin_1,promo_type_2,promo_bin_2,promo_discount_2,promo_discount_type_2
0,1,S0002,P0001,2017-01-02,0.0,0.00,8.0,6.25,PR14,NaN,PR03,NaN,NaN,NaN
1,2,S0002,P0005,2017-01-02,0.0,0.00,11.0,33.90,PR14,NaN,PR03,NaN,NaN,NaN
2,3,S0002,P0011,2017-01-02,0.0,0.00,9.0,49.90,PR14,NaN,PR03,NaN,NaN,NaN
3,4,S0002,P0015,2017-01-02,1.0,2.41,19.0,2.60,PR14,NaN,PR03,NaN,NaN,NaN
4,5,S0002,P0017,2017-01-02,0.0,0.00,12.0,1.49,PR14,NaN,PR03,NaN,NaN,NaN


In [ ]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8886058 entries, 0 to 8886057
Data columns (total 14 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   Unnamed: 0             int64  
 1   store_id               object 
 2   product_id             object 
 3   date                   object 
 4   sales                  float64
 5   revenue                float64
 6   stock                  float64
 7   price                  float64
 8   promo_type_1           object 
 9   promo_bin_1            object 
 10  promo_type_2           object 
 11  promo_bin_2            object 
 12  promo_discount_2       float64
 13  promo_discount_type_2  object 
dtypes: float64(5), int64(1), object(8)
memory usage: 949.1+ MB


In [ ]:
# Change data types
sales_cleaned = pd.DataFrame(sales)
# Convert the column to datetime with the correct format
sales_cleaned['date'] = pd.to_datetime(sales_cleaned['date'], format='%Y-%m-%d')

promo_columns = ['promo_type_1', 'promo_bin_1', 'promo_type_2', 'promo_bin_2', 'promo_discount_2', 'promo_discount_type_2']
sales_cleaned[promo_columns] = sales_cleaned[promo_columns].astype('category')

sales_cleaned.info()
sales_cleaned.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8886058 entries, 0 to 8886057
Data columns (total 14 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   Unnamed: 0             int64         
 1   store_id               object        
 2   product_id             object        
 3   date                   datetime64[ns]
 4   sales                  float64       
 5   revenue                float64       
 6   stock                  float64       
 7   price                  float64       
 8   promo_type_1           category      
 9   promo_bin_1            category      
 10  promo_type_2           category      
 11  promo_bin_2            category      
 12  promo_discount_2       category      
 13  promo_discount_type_2  category      
dtypes: category(6), datetime64[ns](1), float64(4), int64(1), object(2)
memory usage: 593.2+ MB


,Unnamed: 0,store_id,product_id,date,sales,revenue,stock,price,promo_type_1,promo_bin_1,promo_type_2,promo_bin_2,promo_discount_2,promo_discount_type_2
0,1,S0002,P0001,2017-01-02,0.0,0.00,8.0,6.25,PR14,NaN,PR03,NaN,NaN,NaN
1,2,S0002,P0005,2017-01-02,0.0,0.00,11.0,33.90,PR14,NaN,PR03,NaN,NaN,NaN
2,3,S0002,P0011,2017-01-02,0.0,0.00,9.0,49.90,PR14,NaN,PR03,NaN,NaN,NaN
3,4,S0002,P0015,2017-01-02,1.0,2.41,19.0,2.60,PR14,NaN,PR03,NaN,NaN,NaN
4,5,S0002,P0017,2017-01-02,0.0,0.00,12.0,1.49,PR14,NaN,PR03,NaN,NaN,NaN


In [ ]:
# saves sales cleaned file

# sales_cleaned.to_csv('sales_cleaned.csv', index=False)
# sales_cleaned.to_parquet('sales_cleaned.parquet', engine='pyarrow', index=False)

# read parquet file to confirm saved dtypes
# sales_cleaned = pd.read_parquet('/content/drive/Shareddrives/Projeto II/Created_Tables/sales_cleaned.parquet')
# sales_cleaned
# sales_cleaned.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8886058 entries, 0 to 8886057
Data columns (total 14 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   Unnamed: 0             int64         
 1   store_id               object        
 2   product_id             object        
 3   date                   datetime64[ns]
 4   sales                  float64       
 5   revenue                float64       
 6   stock                  float64       
 7   price                  float64       
 8   promo_type_1           category      
 9   promo_bin_1            category      
 10  promo_type_2           category      
 11  promo_bin_2            category      
 12  promo_discount_2       float64       
 13  promo_discount_type_2  category      
dtypes: category(5), datetime64[ns](1), float64(5), int64(1), object(2)
memory usage: 652.5+ MB


##ForecastcRevenue

In [ ]:
fore_rev.head()

,Store,Date,fcst_revenue,WEEk
0,S0002,10/1/2019,739.0,40
1,S0002,10/2/2019,632.0,40
2,S0002,10/3/2019,597.0,40
3,S0002,10/4/2019,624.0,40
4,S0002,10/5/2019,999.0,40


In [ ]:
fore_rev_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1943 entries, 0 to 1942
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Store         1943 non-null   object 
 1   Date          1943 non-null   object 
 2   fcst_revenue  1943 non-null   float64
 3   WEEk          1943 non-null   int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 60.8+ KB


In [ ]:
# change column name WEEK

fore_rev.rename(columns={'WEEk': 'Week'}, inplace=True)

# change dtype of Date column
fore_rev_cleaned = pd.DataFrame(fore_rev)
fore_rev_cleaned['Date'] = pd.to_datetime(fore_rev['Date'])
fore_rev_cleaned['Date'] = fore_rev_cleaned['Date'].dt.strftime('%d/%m/%Y')


fore_rev_cleaned.head()
fore_rev_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1943 entries, 0 to 1942
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Store         1943 non-null   object 
 1   Date          1943 non-null   object 
 2   fcst_revenue  1943 non-null   float64
 3   Week          1943 non-null   int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 60.8+ KB


In [ ]:
# save forecast_revenue cleaned file

# fore_rev_cleaned.to_csv('fore_rev_cleaned.csv', index=False)
# fore_rev_cleaned.to_parquet('fore_rev_cleaned.parquet', engine='pyarrow', index=False)
